In [1]:
import xarray as xr
from bokeh.plotting import output_file, save, output_notebook
from core_portrait_plot_custom import portrait_plot
import numpy as np

In [2]:
# Enable Bokeh output in the notebook
output_notebook()

Loading BokehJS ...

In [3]:
mdl_list = ['CanESM5', 'CMCC-CM2-SR5', 'CNRM-ESM2-1', 'EC-Earth3', 'HadGEM3-GC31-MM', 'IPSL-CM6A-LR', 'MIROC6', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorCPM1']

filename1 = "/global/cfs/projectdirs/m4581/jungchoi/PMP/GMD2026/Fig6/Fig6.Trend_bias.SIE_Arctic.nc"
filename2 = "/global/cfs/projectdirs/m4581/jungchoi/PMP/GMD2026/Fig6/Fig6.Trend_bias.SIE_Antarctic.nc"

ds1 = xr.open_dataset(filename1)
ds2 = xr.open_dataset(filename2)

var1_data = ds1["sic"].values
var2_data = ds2["sic"].values

In [4]:
region1_index = range(0, 30, 3)
region2_index = range(1, 30, 3)
region3_index = range(2, 30, 3)

In [5]:
central_arctic_data = []
for ca in region1_index:
    central_arctic_data.append(var1_data[ca])

north_atlantic_data = []
for na in region2_index:
    north_atlantic_data.append(var1_data[na])

north_pacific_data = []
for np in region3_index:
    north_pacific_data.append(var1_data[np])

indian_ocean_data = []
for io in region1_index:
    indian_ocean_data.append(var2_data[io])
    
south_atlantic_data = []
for sa in region2_index:
    south_atlantic_data.append(var2_data[sa])
    
south_pacific_data = []
for sp in region3_index:
    south_pacific_data.append(var2_data[sp])

In [6]:
xaxis_labels = ['LY1', 'LY2', 'LY3', 'LY4', 'LY5', 'LY6', 'LY7', 'LY8', 'LY9', 'LY10', 'HIST']
yaxis_labels = mdl_list
cmap_arctic = [-24, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20, 24]
cmap_antarctic = [-50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50]

In [7]:
p = portrait_plot(
    [central_arctic_data, north_atlantic_data, north_pacific_data][::-1], 
    static=True,
    xaxis_labels=xaxis_labels,
    yaxis_labels=yaxis_labels,
    xaxis_fontsize=13,
    yaxis_fontsize=13,
    height=1000, width=1000,
    cmap="RdBu_r",
    cmap_bounds=cmap_arctic,
    title="Arctic Sea Ice Extent Trend bias",
    show_plot=False
    )

In [8]:
# Adding some customizations
from bokeh.plotting import figure, show
from bokeh.models import Span, Title, Div
from bokeh.io import output_file, save
from bokeh.layouts import column, row

line_locations = range(0, 12)

for loc in line_locations:
    span = Span(location=loc, dimension='width', line_color='black', line_width=2)
    p.add_layout(span)
    
for loc in line_locations:
    span = Span(location=loc, dimension='height', line_color='black', line_width=0.5)
    p.add_layout(span)
    
legend_title = Title(text="[10⁶ · km²]", align="center", text_font_size="13pt")
p.add_layout(legend_title, 'right')
p.title.text_font_size = '14pt'
show(p)

In [9]:
img_x_labels = ['LY1', 'LY2', 'LY3', 'LY4', 'LY5', 'LY6', 'LY7', 'LY8', 'LY9', 'LY10', 'HIST']

In [10]:
# custom function that creates an array of img URLs for each glyph, with custom sort
# use xaxis labels and mdl_list
def get_img_urls(region: str='Arctic'):
    urls = []
    url_head = f'https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/dcpp/trend_bias/Trend_SIC.{region}'
    for n in range(3):
        for mdl in mdl_list[::-1]:
            for xlbl in img_x_labels:
                url = f'{url_head}.{mdl}.{xlbl}.png'
                urls.append(url)
    return urls

In [11]:
arctic_img_urls = get_img_urls(region='Arctic')
antarctic_img_urls = get_img_urls(region='Antarctic')

In [12]:
# creating interactive version
p = portrait_plot(
    [central_arctic_data, north_atlantic_data, north_pacific_data][::-1],
    img_url = arctic_img_urls,
    clickable = True,
    xaxis_labels=xaxis_labels,
    yaxis_labels=yaxis_labels,
    xaxis_fontsize=13,
    yaxis_fontsize=13,
    height=1000, width=1000,
    cmap="RdBu_r",
    vrange=(-24,24),
    title="Arctic Sea Ice Extent Trend bias (ref: HadISST_ICE 1979-2014)",
    show_plot=False
    )

line_locations = range(0, 12)

for loc in line_locations:
    h_lines = Span(location=loc, dimension='width', line_color='black', line_width=3)
    p.add_layout(h_lines)
    
for loc in line_locations:
    v_lines = Span(location=loc, dimension='height', line_color='black', line_width=0.7)
    p.add_layout(v_lines)
    
end_line = Span(location=10.99, dimension='height', line_color='black', line_width=0.7)
p.add_layout(end_line)

hist_line = Span(location=10, dimension='height', line_color='black', line_width=3)
p.add_layout(hist_line)
    
legend_title = Title(text="[10⁴ km² per 10 yr]", align="center", text_font_size="13pt")
p.add_layout(legend_title, 'right')
p.title.text_font_size = '14pt'

# Add text to top of the page
text_title = """
<div class=title>
<center>
<h1>CMIP6 DCPP Arctic Sea Ice Extent Trend bias</h1>
</center>
</div>
"""
logos = """
<div class=title>
<center>
<br>

<a href="https://pcmdi.llnl.gov/" target="_blank">
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_PCMDI.png" height="50" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="PCMDI"
    title="PCMDI"></a>

<a href=https://climatemodeling.science.energy.gov/program/regional-global-model-analysis target=_blank>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_doe.png height="45" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="US Sponsor: DOE BER RGMA"
    title="US Sponsor: DOE BER RGMA"></a>

</center>
</div>
<br>
"""

text_top = text_title + logos
text_top += """
<style>
.title {
    width: 160%;
}
.box1 {
    background-color: #E8E8E8;
    width: 160%;
    padding: 10px;
    margin: 0px;
}
.dropdown {
    background-color: white;
    width: 100%;
    padding: 10px;
    margin: 0px;
}
</style>

<div class=box1>
<p><b><font size=2 color=#e67e22>Supported browsers: Chrome, Firefox, Safari, Microsoft Edge</font></b></p>

<p><b><font size=4 color=darkblue> Reference and Contributions </font></b><br>
- Citation: Choi et al. (2026, in prep.)<br>
- The calculation has been conducted and dive down plots were generated by Jung Choi.<br>
- The Interactive Portrait Plot and webpage are generated by <a href=https://people.llnl.gov/chang61 target=_blank>Kristin Chang</a> and <a href=https://pcmdi.llnl.gov/staff/lee/ target=_blank>Jiwoo Lee</a> (PCMDI/LLNL).<br>
"""

overall_text_usage = """
<p><b><font size=4 color=darkblue> Usage</b> <i>(Important for navigating to dive-down diagnostics)</i></font><br>
- Hover your mouse over boxes will show tooltips for metric values and preview of dive down plots.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/tap.svg style="height: 15px;">: Turn on/off <b>Tap</b> icon to enable/disable clicking on boxes.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/save.svg style="height: 15px;">: Click <b>Save</b> icon to download a static HTML file of the plot.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/hover.svg style="height: 15px;">: Turn on/off <b>Hover</b> icon enable/disable showing tooltips.<br>
"""

legend_img = """
<style>
h3 {
  float:right;
  margin: 10px -330px;
}
</style>
<h3>Region Legend</h3>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/dcpp/mean_bias/arctic_legend.png height="55"
    style="vertical-align:middle;horizontal-align:right;margin:40px -400px;float:right;"
    alt="PCMDI Mean Bias Region Legend"
    title="PCMDI Mean Bias Region Legend">
"""

text_top += overall_text_usage + "</div>" + legend_img

# LLNL Disclaimer
#

text_bottom = """
<style>
.bottom {
    width: 1000px;
}
.fig_desc {
    width:1000px;
    font-size:11pt;
}
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
}
td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
    text-align: center;
}
tr:nth-child(even) {
    background-color: #dddddd;
}
</style>
"""

text_bottom += """
<div class=fig_desc>
<p><b><font size=4 color=darkblue> Figure Description</b></font><br>
This portrait plot shows the trend bias in sea ice extent for each model over the period 1979-2014. First, area averages are taken for three subregions of the Arctic for each month. The three subregions are defined as in <a href=https://journals.ametsoc.org/view/journals/clim/29/24/jcli-d-16-0026.1.xml>Ivanova et al. (2016)</a> <a href=https://pcmdi.github.io/pcmdi_metrics/_images/examples_Demo_9_seaIceExtent_ivanova_12_0.png>[see the map]</a>. 
Then, the annual averages are calculated as a function of the lead year (LY). The rightmost column shows the mean bias from historical experiments (HIST) for the same period. HadISST_ICE is used as the reference dataset. Units are [10<sup>6</sup> km<sup>2</sup>]. The interactive subplots show the predicted linear trend of sea ice concentrations in shading, which are produced by averaging data over two-month periods. Red (purple) lines represent the observed trends of 8 and 16% (–8 and –16%, respectively) per 10 years from HadISST_ICE for the same period.
</div>
<br><br>
<div class=bottom>
<br><p>
<a href="https://www.llnl.gov" target=_blank>
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_llnl-600.png" 
alt="LLNL" style="float:left;height:80px;margin-right:10px">
</a>
The work of PCMDI and results presented on this web page are supported by the <a href=https://eesm.science.energy.gov/program-area/regional-global-model-analysis>Regional and Global Model Analysis (RGMA) program area
</a> under the <a href=https://eesm.science.energy.gov/about>Earth and Environmental System Modeling (EESM) program
</a> within the <a href=https://science.osti.gov/ber/Research/eessd>Earth and Environmental Systems Sciences Division (EESSD)
</a> of the <a href=https://science.osti.gov/>United States Department of Energy's (DOE) Office of Science (OSTI)
</a>. The work of PCMDI is performed under the auspices of the U.S. DOE by <a href=https://pls.llnl.gov/>Lawrence Livermore National Laboratory (LLNL)
</a> under contract DE-AC52-07NA27344. LLNL-WEB-812310.</p>
</div>
"""

# combine page
page = column(Div(text=text_top),
              p,
              Div(text=text_bottom)) 
    
#show(p)
output_file(filename="interactive_Arctic_trend_bias_portrait_plot.html", title="Interactive Arctic Sea Ice Trend Bias Portrait Plot")
save(page, filename='./interactive_Arctic_trend_bias_portrait_plot.html')

'/global/cfs/cdirs/m4581/jungchoi/PMP/interactive_Mean_bias_portrait/interactive_Arctic_trend_bias_portrait_plot.html'

In [13]:
# creating interactive version
p = portrait_plot(
    [indian_ocean_data, south_atlantic_data, south_pacific_data][::-1],
    img_url = antarctic_img_urls,
    clickable = True,
    xaxis_labels=xaxis_labels,
    yaxis_labels=yaxis_labels,
    xaxis_fontsize=13,
    yaxis_fontsize=13,
    height=1000, width=1000,
    cmap="RdBu_r",
    vrange=(-40, 40),
    title="Antarctic Sea Ice Extent Trend bias (ref: HadISST_ICE 1979-2014)",
    show_plot=False
    )

line_locations = range(0, 12)

for loc in line_locations:
    h_lines = Span(location=loc, dimension='width', line_color='black', line_width=3)
    p.add_layout(h_lines)
    
for loc in line_locations:
    v_lines = Span(location=loc, dimension='height', line_color='black', line_width=0.7)
    p.add_layout(v_lines)
    
end_line = Span(location=10.99, dimension='height', line_color='black', line_width=0.7)
p.add_layout(end_line)

hist_line = Span(location=10, dimension='height', line_color='black', line_width=3)
p.add_layout(hist_line)
    
legend_title = Title(text="[10⁴ km² per 10 yr]", align="center", text_font_size="13pt")
p.add_layout(legend_title, 'right')
p.title.text_font_size = '14pt'

# Add text to top of the page
text_title = """
<div class=title>
<center>
<h1>CMIP6 DCPP Antarctic Sea Ice Extent Trend bias</h1>
</center>
</div>
"""
logos = """
<div class=title>
<center>
<br>

<a href="https://pcmdi.llnl.gov/" target="_blank">
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_PCMDI.png" height="50" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="PCMDI"
    title="PCMDI"></a>

<a href=https://climatemodeling.science.energy.gov/program/regional-global-model-analysis target=_blank>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_doe.png height="45" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="US Sponsor: DOE BER RGMA"
    title="US Sponsor: DOE BER RGMA"></a>

</center>
</div>
<br>
"""

text_top = text_title + logos
text_top += """
<style>
.title {
    width: 160%;
}
.box1 {
    background-color: #E8E8E8;
    width: 160%;
    padding: 10px;
    margin: 0px;
}
.dropdown {
    background-color: white;
    width: 100%;
    padding: 10px;
    margin: 0px;
}
</style>

<div class=box1>
<p><b><font size=2 color=#e67e22>Supported browsers: Chrome, Firefox, Safari, Microsoft Edge</font></b></p>

<p><b><font size=4 color=darkblue> Reference and Contributions </font></b><br>
- Citation: Choi et al. (2026, in prep.)<br>
- The calculation has been conducted and dive down plots were generated by Jung Choi.<br>
- The Interactive Portrait Plot and webpage are generated by <a href=https://people.llnl.gov/chang61 target=_blank>Kristin Chang</a> and <a href=https://pcmdi.llnl.gov/staff/lee/ target=_blank>Jiwoo Lee</a> (PCMDI/LLNL).<br>
"""

overall_text_usage = """
<p><b><font size=4 color=darkblue> Usage</b> <i>(Important for navigating to dive-down diagnostics)</i></font><br>
- Hover your mouse over boxes will show tooltips for metric values and preview of dive down plots.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/tap.svg style="height: 15px;">: Turn on/off <b>Tap</b> icon to enable/disable clicking on boxes.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/save.svg style="height: 15px;">: Click <b>Save</b> icon to download a static HTML file of the plot.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/hover.svg style="height: 15px;">: Turn on/off <b>Hover</b> icon enable/disable showing tooltips.<br>
"""

legend_img = """
<style>
h3 {
  float:right;
  margin: 10px -330px;
}
</style>
<h3>Region Legend</h3>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/dcpp/mean_bias/antarctic_legend.png height="55"
    style="vertical-align:middle;horizontal-align:right;margin:40px -400px;float:right;"
    alt="PCMDI Trend Bias Region Legend"
    title="PCMDI Trend Bias Region Legend">
"""

text_top += overall_text_usage + "</div>" + legend_img

# LLNL Disclaimer
#

text_bottom = """
<style>
.bottom {
    width: 1000px;
}
.fig_desc {
    width:1000px;
    font-size:11pt;
}
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
}
td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
    text-align: center;
}
tr:nth-child(even) {
    background-color: #dddddd;
}
</style>
"""

text_bottom += """
<div class=fig_desc>
<p><b><font size=4 color=darkblue> Figure Description</b></font><br>
This portrait plot shows the trend bias in sea ice extent for each model over the period 1979-2014. First, area averages are taken for three subregions of the Antarctic for each month. The three subregions are defined as in <a href=https://journals.ametsoc.org/view/journals/clim/29/24/jcli-d-16-0026.1.xml>Ivanova et al. (2016)</a> <a href=https://pcmdi.github.io/pcmdi_metrics/_images/examples_Demo_9_seaIceExtent_ivanova_12_1.png>[see the map]</a>. 
Then, the annual averages are calculated as a function of the lead year (LY). The rightmost column shows the mean bias from historical experiments (HIST) for the same period. HadISST_ICE is used as the reference dataset. Units are [10<sup>6</sup> km<sup>2</sup>]. The interactive subplots show the predicted linear trend of sea ice concentrations in shading, which are produced by averaging data over two-month periods. Red (purple) lines represent the observed trends of 8 and 16% (–8 and –16%, respectively) per 10 years from HadISST_ICE for the same period.
</div>
<br><br>
<div class=bottom>
<br><p>
<a href="https://www.llnl.gov" target=_blank>
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_llnl-600.png" 
alt="LLNL" style="float:left;height:80px;margin-right:10px">
</a>
The work of PCMDI and results presented on this web page are supported by the <a href=https://eesm.science.energy.gov/program-area/regional-global-model-analysis>Regional and Global Model Analysis (RGMA) program area
</a> under the <a href=https://eesm.science.energy.gov/about>Earth and Environmental System Modeling (EESM) program
</a> within the <a href=https://science.osti.gov/ber/Research/eessd>Earth and Environmental Systems Sciences Division (EESSD)
</a> of the <a href=https://science.osti.gov/>United States Department of Energy's (DOE) Office of Science (OSTI)
</a>. The work of PCMDI is performed under the auspices of the U.S. DOE by <a href=https://pls.llnl.gov/>Lawrence Livermore National Laboratory (LLNL)
</a> under contract DE-AC52-07NA27344. LLNL-WEB-812310.</p>
</div>
"""

# combine page
page = column(Div(text=text_top),
              p,
              Div(text=text_bottom)) 
    
#show(p)
output_file(filename="interactive_Antarctic_trend_bias_portrait_plot.html", title="Interactive Antarctic Sea Ice Trend Bias Portrait Plot")
save(page, filename='./interactive_Antarctic_trend_bias_portrait_plot.html')

'/global/cfs/cdirs/m4581/jungchoi/PMP/interactive_Mean_bias_portrait/interactive_Antarctic_trend_bias_portrait_plot.html'